In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def black_box_bench(x, *, dim_block=5, global_bias=0.0):
    def black_box_bench(x, *, dim_block=5, global_bias=0.0):
    x = np.asarray(x, dtype=float).ravel()
    assert x.size == 3*dim_block, f"Expected length {3*dim_block}, got {x.size}"

    def rastrigin(z):
        A = 10.0; z = np.asarray(z); n = z.size
        return A*n + np.sum(z*z - A*np.cos(2*np.pi*z))
    def ackley(z):
        z = np.asarray(z); n = z.size
        s1 = np.sum(z*z); s2 = np.sum(np.cos(2*np.pi*z))
        return -20*np.exp(-0.2*np.sqrt(s1/n)) - np.exp(s2/n) + 20 + math.e
    def rosenbrock(z):
        z = np.asarray(z); 
        return np.sum(100.0*(z[1:] - z[:-1]**2)**2 + (1.0 - z[:-1])**2)

    def make_R(n, seed):
        rng = np.random.default_rng(seed)
        A = rng.normal(size=(n, n))
        Q, R = np.linalg.qr(A)
        d = np.sign(np.diag(R)); d[d==0] = 1.0
        Q = Q * d
        if np.linalg.det(Q) < 0: Q[:,0] = -Q[:,0]
        return Q

    R1 = make_R(dim_block, seed=101)
    R2 = make_R(dim_block, seed=202)
    R3 = make_R(dim_block, seed=303)

    xopt1 = np.full(dim_block,  1.0)
    xopt2 = np.full(dim_block, -1.5)
    xopt3 = np.full(dim_block,  0.5)

    x1 = x[0:dim_block]
    x2 = x[dim_block:2*dim_block]
    x3 = x[2*dim_block:3*dim_block]

    def penalty(xb, lo, hi):
        if np.all((xb >= lo) & (xb <= hi)): return 0.0
        below = np.clip(lo - xb, 0, None)
        above = np.clip(xb - hi, 0, None)
        return 1e6 * (1.0 + float(np.sum(below*below + above*above)))

    pen  = penalty(x1, -5.12, 5.12) + penalty(x2, -32.768, 32.768) + penalty(x3, -5.0, 10.0)
    if pen > 0:
        print("Out-of-domain penalty applied.")
        print("Domains: [-5.12,5.12] ; [-32.768,32.768] ; [-5,10]")
        print("Returning global_bias + penalty.")
        return float(global_bias + pen)

    z1 = R1 @ (x1 - xopt1); f1 = rastrigin(z1)
    z2 = R2 @ (x2 - xopt2); f2 = ackley(z2)
    z3 = R3 @ (x3 - xopt3) + 1.0; f3 = rosenbrock(z3)

    #print("f1 (Rastrigin):", float(f1), "f2 (Ackley):", float(f2), "f3 (Rosenbrock):", float(f3))
    return float(f1 + f2 + f3 + global_bias)
    pass

In [ ]:
max_evals = 100_000  # MUST use this identifier!
bounds = [
    (-5.12, 5.12),      # dims 0-4: Rastrigin
    (-32.768, 32.768),  # dims 5-9: Ackley
    (-5.0, 10.0)        # dims 10-14: Rosenbrock
]
bounds_15d = (bounds[0] * 5) + (bounds[1] * 5) + (bounds[2] * 5)

In [ ]:
eval_count = 0
def fitness(x):
    global eval_count
    eval_count += 1
    return black_box_bench(x)

In [ ]:
seed = 42  # IMPORTANT: Record this for reproducibility!
np.random.seed(seed)



In [ ]:
print(f"Best X: {best_x}")
print(f"Best f(X): {best_fx}")
print(f"Total evaluations: {eval_count}")
print(f"Seed used: {seed}")

## Visualization